In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [10]:
data = pd.read_csv(r'D:\Deep_Learning_Projects\Churn_prediction_ANN\Data\Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [12]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

## **Basic preprocessing**

In [13]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [14]:
## Label Encoding for Gender Column
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data['Gender'].value_counts()
print(label_encoder.classes_)

['Female' 'Male']


In [15]:
## One hot encoding for Geography column

one_hot_encoder = OneHotEncoder()
geography_encoded = one_hot_encoder.fit_transform(data[['Geography']])
geography_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [16]:
one_hot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [17]:
geography_encoded_df = pd.DataFrame(data = geography_encoded.toarray(), columns=one_hot_encoder.get_feature_names_out(['Geography']))
geography_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [18]:
## Combine the geography one hot encoded df with original df

data = pd.concat([data.drop('Geography', axis= 1), geography_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [19]:
## Save the encoders

with open('gender_label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)


with open('geography_one_hot_encoder.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder, file)

In [20]:
## Split the dataset

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=462)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
X_train.shape

(7000, 12)

In [22]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## **ANN Implementation**

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [24]:
## Define the ANN structure

model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1], )),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

d:\Deep_Learning_Projects\Churn_prediction_ANN\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
## Compile the ANN

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
## Setup the tensorboard

log_dir = "logs/model/" + str(datetime.datetime.now().date())
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [28]:
## Setup the early stopping to break the epoch at any point of time

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [29]:
## Train the ANN

history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), 
    epochs=100, callbacks=[tensorflow_callback, early_stopping]
    )

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8183 - loss: 0.4423 - val_accuracy: 0.8540 - val_loss: 0.3436
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8504 - loss: 0.3626 - val_accuracy: 0.8623 - val_loss: 0.3401
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8573 - loss: 0.3543 - val_accuracy: 0.8617 - val_loss: 0.3440
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8589 - loss: 0.3433 - val_accuracy: 0.8630 - val_loss: 0.3263
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8568 - loss: 0.3439 - val_accuracy: 0.8650 - val_loss: 0.3228
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8609 - loss: 0.3362 - val_accuracy: 0.8603 - val_loss: 0.3378
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8613 - loss: 0.3385 - val_accuracy: 0.8677 - val_loss: 0.3298
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8602 - loss: 0.3354 - val_accu

In [30]:
## Save the model 

model.save('chrun_classifier.h5')

In [31]:
## Load tensorboard extension

%load_ext tensorboard

In [32]:
%tensorboard --logdir logs/model/

Reusing TensorBoard on port 6006 (pid 10148), started 3 days, 20:16:04 ago. (Use '!kill 10148' to kill it.)

## **Prediction using above trained ANN saved models**

In [33]:
from tensorflow.keras.models import load_model

model = load_model('D:\Deep_Learning_Projects\Churn_prediction_ANN\chrun_classifier.h5')

with open('D:\Deep_Learning_Projects\Churn_prediction_ANN\gender_label_encoder.pkl', 'rb') as file:
    gender_encoder = pickle.load(file)

print(gender_encoder.classes_)

with open('D:\Deep_Learning_Projects\Churn_prediction_ANN\geography_one_hot_encoder.pkl', 'rb') as file:
    geography_encoder = pickle.load(file)

with open('D:\Deep_Learning_Projects\Churn_prediction_ANN\scaler.pkl', 'rb') as file:
    scaler_model = pickle.load(file)

['Female' 'Male']


In [34]:
geography_encoder.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [35]:
input_data = {
    'CreditScore': 404,
    'Geography': 'Spain', 
    'Gender' : 'Female',
    'Age': 26,
    'Tenure': 5,
    'Balance': 638633,
    'NumOfProducts': 3,
    'HasCrCard': 1,
    'IsActiveMember': 0,
    'EstimatedSalary': 63853	
}

input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,404,Spain,Female,26,5,638633,3,1,0,63853


In [36]:
input_geo_encoded = geography_encoder.transform([input_data['Geography']]).toarray()
input_geo_encoded_df = pd.DataFrame(input_geo_encoded, columns=geography_encoder.get_feature_names_out(['Geography']))
input_geo_encoded_df

d:\Deep_Learning_Projects\Churn_prediction_ANN\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [37]:
input_data = pd.concat([input_data.drop('Geography', axis=1), input_geo_encoded_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,404,Female,26,5,638633,3,1,0,63853,0.0,0.0,1.0


In [38]:
input_data['Gender']= gender_encoder.transform(input_data['Gender'])
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,404,0,26,5,638633,3,1,0,63853,0.0,0.0,1.0


In [39]:
input_data = scaler_model.transform(input_data)
input_data

array([[-2.53860968e+00, -1.08316117e+00, -1.24325088e+00,
        -5.88788054e-03,  8.97949439e+00,  2.55224817e+00,
         6.46647006e-01, -1.01642051e+00, -6.29734421e-01,
        -1.01554945e+00, -5.74710940e-01,  1.76021608e+00]])

## **Predict the churn based on above data**

In [44]:
prediction = model.predict(input_data)
prediction[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


0.99662644

**The prediction probability is 0.99 which is close to 1, that means this customer is most likely to churn.**